In [1]:
import os; import numpy
from gasp3.sql.fm import Q_to_df
from gasp3.adv.txtcls.cls import text_prediction
from gasp3.adv.txtcls.eval import binary_eval
from gasp3.adv.txtcls.freq import correlated_words
from gasp3.pyt.df.stats import df_to_freqdf

In [2]:
conDB = {
    'HOST' : 'localhost', 'PORT' : '5432', 'USER' : 'postgres',
    'PASSWORD' : 'admin', 'DATABASE' : 'expvgim_db'
}

TBL_SCHEMA = {
    'SAMPLE_T' : 'geotmlnh_facesample',
    'SAMPLE_FK' : 'postid',
    'DATA_T'   : 'geotmlnh_facedata',
    'DATA_ID'  : 'post_id',
    'TXT_COL'  : (
        "CASE WHEN geotmlnh_facedata.type = 'link' "
        "THEN lower(unaccent(geotmlnh_facedata.description)) "
        "ELSE lower(unaccent(geotmlnh_facedata.message)) END"
    ),
    'TIME_COL' : 'datahora',
    'REF_COL'  : 'is_fire'
}

TRAIN_DIM = 80

OUT_FLD = r'F:\geotmlnh\text_cls'

In [ ]:
"""
Sanitize data to use in classification
"""

Q1 = (
    "(SELECT *, to_tsvector('portuguese', regexp_replace("
        "regexp_replace(lower(unaccent(txtcls)), 'http://[^:\s]+(\S+)', "
        "' ', 'g'), '[^\w]+', ' ', 'g')) "
    "AS txtsan FROM ("
        "SELECT {t}.fid, {t}.{t_id}, {txtcol} AS txtcls, "
        "{dt}.{timecol} AS datahora, {t}.{refcol} "   
        "FROM {t} INNER JOIN {dt} ON "
        "{t}.{t_id} = {dt}.{dt_id}"
    ") AS foo) AS stop_table"
).format(
    txtcol=TBL_SCHEMA['TXT_COL'], t=TBL_SCHEMA['SAMPLE_T'],
    dt=TBL_SCHEMA['DATA_T'], t_id=TBL_SCHEMA['SAMPLE_FK'],
    dt_id=TBL_SCHEMA['DATA_ID'], timecol=TBL_SCHEMA['TIME_COL'],
    refcol=TBL_SCHEMA['REF_COL']
)

cols = ['fid', TBL_SCHEMA['TIME_COL'], TBL_SCHEMA['SAMPLE_FK'], TBL_SCHEMA['REF_COL']]
Q2 = (
    "SELECT {selCols}, ARRAY_TO_STRING(array_agg("
        "word ORDER BY word_index), ' ', '*') AS txtsan, "
    "REPLACE(CAST(TRIM("
        "stop_table.txtcls) AS text), '''', '') AS no_duplicated "
    "FROM ("
        "SELECT fid, word, CAST(UNNEST(word_index) AS integer) "
        "AS word_index FROM ("
            "SELECT fid, SPLIT_PART(tst, ';', 1) AS word, "
            "STRING_TO_ARRAY(SPLIT_PART(tst, ';', 2), ',') AS word_index FROM ("
                "SELECT fid, REPLACE(REPLACE(REPLACE(REPLACE(REPLACE("
                    "CAST(UNNEST(txtsan) AS text), "
                        "',{{', ',\"{{'), ',\"{{', ';'), '}}\"', ''), "
                        "'(', ''), '}}', '') AS tst "
                "FROM {tbl}"
            ") AS foo"
        ") AS foo2"
    ") AS foo3 INNER JOIN {tbl} ON foo3.fid = stop_table.fid "
    "GROUP BY {selCols}, stop_table.txtcls"
).format(
    tbl=Q1, selCols=", ".join(["stop_table.{}".format(i) for i in cols])
)

In [ ]:
dt_ref = Q_to_df(conDB, Q2)

ref_Count = df_to_freqdf(dt_ref, TBL_SCHEMA['REF_COL'])

In [ ]:
if TRAIN_DIM == 100:
    dt_train = dt_ref
    dt_cls   = dt_ref.copy()

else:
    ref_Count['n_train'] = (ref_Count['count'] * TRAIN_DIM) / 100
    ref_Count = ref_Count.round({'n_train' : 0})
    
    dt_ref['RN'] = dt_ref.sort_values(
        [TBL_SCHEMA['REF_COL']], ascending=[True]
    ).groupby([TBL_SCHEMA['REF_COL']]).cumcount() + 1
    
    for idx, row in ref_Count.iterrows():
        if idx == 0:
            dt_ref['is_train'] = numpy.where(
                (dt_ref['is_fire'] == row.is_fire) & (dt_ref.RN <= row.n_train),
                1, 0
            )
        else:
            dt_ref['is_train'] = numpy.where(
                (dt_ref['is_fire'] == row.is_fire) & (dt_ref.RN <= row.n_train),
                1, dt_ref.is_train
            )
    
    dt_cls   = dt_ref[dt_ref.is_train==0]
    dt_train = dt_ref[dt_ref.is_train==1]

In [ ]:
mdls = {
    'NB' : 'NaiveBayes', 'SVM' :'LinearSupportVectorMachine',
    'RF' : 'RandomForest', 'LOGREG':'LogisticRegression'}

corr_words = correlated_words(
    dt_train, TBL_SCHEMA['REF_COL'], 'txtsan',
    os.path.join(OUT_FLD, 'corr_words_{}.xlsx'.format(str(TRAIN_DIM))),
    lang='english'
)

for m in mdls:
    tbl_cls = text_prediction(
        dt_train, dt_cls, TBL_SCHEMA['REF_COL'], 'txtsan', 'txtsan',
        os.path.join(OUT_FLD, 'res_{}_{}.xlsx'.format(m, str(TRAIN_DIM))),
        lang='english', method=mdls[m]
    )
    
    mtx = binary_eval(
        tbl_cls, TBL_SCHEMA['SAMPLE_FK'], TBL_SCHEMA['REF_COL'],
        tbl_cls, TBL_SCHEMA['SAMPLE_FK'],
        os.path.join(OUT_FLD, 'mtx_{}_{}.xlsx'.format(m, str(TRAIN_DIM))),
        tstCol = 'classification'
    )

In [3]:
from gasp3.adv.txtcls.cls import get_rows_related_with_event
from gasp3.to import db_to_tbl

QT = (
    "(SELECT {t}.fid, {t}.{t_id}, {txtcol} AS txtcls, "
    "{dt}.{timecol} AS datahora, {t}.{refcol} "   
    "FROM {t} INNER JOIN {dt} ON "
    "{t}.{t_id} = {dt}.{dt_id}) AS foo"
).format(
    txtcol=TBL_SCHEMA['TXT_COL'], t=TBL_SCHEMA['SAMPLE_T'],
    dt=TBL_SCHEMA['DATA_T'], t_id=TBL_SCHEMA['SAMPLE_FK'],
    dt_id=TBL_SCHEMA['DATA_ID'], timecol=TBL_SCHEMA['TIME_COL'],
    refcol=TBL_SCHEMA['REF_COL']
)

tbl_ref = db_to_tbl(
    conDB, "SELECT * FROM {}".format(QT),
    os.path.join(OUT_FLD, 'tbl_ref.xlsx')
)

tbl_res = get_rows_related_with_event(conDB, {
    "TNAME" : QT,
    "TEXTCOL" : "txtcls",
    "TIMECOL" : "datahora",
    "SELCOL"  : ['fid', TBL_SCHEMA['SAMPLE_FK'], "datahora", TBL_SCHEMA['REF_COL']]
}, ['fog', 'incendi'], os.path.join(OUT_FLD, 'res_q.xlsx'))

mtx_q = binary_eval(
    tbl_ref, TBL_SCHEMA['SAMPLE_FK'], TBL_SCHEMA['REF_COL'],
    tbl_res, TBL_SCHEMA['SAMPLE_FK'],
    os.path.join(OUT_FLD, 'mtx_q.xlsx'),
    tstCol = None
)